In [8]:
from func import *

In [9]:
train_df = load_csvs_from_ftp_to_df(provided_local_dir="/")

In [10]:
positive_col_list = ["BedroomsTotal",
                     "BathroomsTotalInteger",
                     "LotSizeAcres",
                     "LotSizeArea",
                     "LotSizeSquareFeet"]
non_negative_col_list = ["ParkingTotal"]
default_drop_col = ["ListingId",
                    "ListingKey",
                    "ListingKeyNumeric",
                    "ListPrice",
                    "OriginalListPrice"]

In [11]:
train_df_clean, knn_model, train_df_ref, k_means_model, col_drop_list, scaler = pre_process(train_df,
                                                                                            price_col="ClosePrice",
                                                                                            default_threshold=0.8,
                                                                                            col_drop=default_drop_col,
                                                                                            positive_col_list=positive_col_list,
                                                                                            non_negative_col_list=non_negative_col_list,
                                                                                            flag_col_list=["Flooring"],
                                                                                            yn_col_list=["AttachedGarageYN",
                                                                                                         "ViewYN",
                                                                                                         "NewConstructionYN",
                                                                                                         "PoolPrivateYN",
                                                                                                         "FireplaceYN"],
                                                                                            knn_k=3,
                                                                                            knn_model=None,
                                                                                            train_df_ref=None,
                                                                                            num_clusters=10,
                                                                                            clustering_method="k-means",
                                                                                            clustering_model=None,
                                                                                            scaler_method="robust",
                                                                                            scaler=None,
                                                                                            save_name="processed",
                                                                                            train_data=True,
                                                                                            save=True
                                                                                            )

C:\My_Programs\Python_Project\IDX_Intern_self\func.py:246: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern_self\func.py:246: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern_self\func.py:246: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beh

In [12]:
test_df = load_csvs_from_ftp_to_df(provided_local_dir="/",
                                   date_range=range(12, 13))
col_drop_list.extend(default_drop_col)

In [13]:
test_df_clean, _, _, _, _, _ = pre_process(test_df,
                               price_col="ClosePrice",
                               default_threshold=0.8,
                               col_drop=col_drop_list,
                               positive_col_list=positive_col_list,
                               non_negative_col_list=non_negative_col_list,
                               flag_col_list=["Flooring"],
                               yn_col_list=["AttachedGarageYN",
                                            "ViewYN",
                                            "NewConstructionYN",
                                             "PoolPrivateYN",
                                             "FireplaceYN"],
                               knn_k=3,
                               knn_model=knn_model,
                               train_df_ref=train_df_ref,
                               num_clusters=10,
                               clustering_method="k-means",
                               clustering_model=k_means_model,
                               scaler_method="robust",
                               scaler=scaler,
                               save_name="processed",
                               train_data=False,
                               save=True
                               )

C:\My_Programs\Python_Project\IDX_Intern_self\func.py:246: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern_self\func.py:246: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern_self\func.py:246: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beh

# Train model (Linear regression)

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

import category_encoders as ce


def mdape(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = y_true != 0
    return np.median(np.abs(y_true[mask] - y_pred[mask]) / np.abs(y_true[mask])) * 100 if mask.any() else np.nan


def fit_mixed_encoders(train_df, test_df, target_col, alpha=10.0,
                       low_card_max=20, high_card_min=200,
                       drop_cols=None):

    drop_cols = drop_cols or []

    train_df = train_df.drop(columns=drop_cols, errors="ignore")
    test_df  = test_df.drop(columns=drop_cols, errors="ignore")

    X_train = train_df.drop(columns=[target_col])
    y_train = train_df[target_col]

    X_test  = test_df.drop(columns=[target_col])
    y_test  = test_df[target_col]

    num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    cat_cols = [c for c in X_train.columns if c not in num_cols]

    # Cardinality on TRAIN only
    nunique = X_train[cat_cols].nunique(dropna=False)

    low_card_cols  = nunique[(nunique <= low_card_max)].index.tolist()
    high_card_cols = nunique[(nunique >= high_card_min)].index.tolist()
    mid_card_cols  = [c for c in cat_cols if c not in low_card_cols + high_card_cols]

    # Encoders
    ohe = OneHotEncoder(handle_unknown="ignore")  # sparse by default
    te  = ce.TargetEncoder(cols=high_card_cols + mid_card_cols,
                           smoothing=10, min_samples_leaf=20)

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(with_mean=False), num_cols),
            ("low_ohe", ohe, low_card_cols),
            ("mid_high_te", te, high_card_cols + mid_card_cols),
        ],
        remainder="drop",
        sparse_threshold=1.0
    )

    model = Pipeline([
        ("prep", preprocessor),
        ("ridge", Ridge(alpha=alpha, random_state=42))
    ])

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    y_pred_true = np.expm1(y_pred)
    y_test_true = np.expm1(y_test)

    return {
        "model": model,
        "groups": {
            "low_card_ohe": low_card_cols,
            "mid_card_te": mid_card_cols,
            "high_card_te": high_card_cols
        },
        "r2": r2_score(y_test_true, y_pred_true),
        "mdape": mdape(y_test_true, y_pred_true),
    }




In [17]:
train_df_clean["logClosePrice"] = np.log1p(train_df_clean["ClosePrice"])
test_df_clean["logClosePrice"] = np.log1p(test_df_clean["ClosePrice"])

In [20]:
res = fit_mixed_encoders(train_df_clean,
                         test_df_clean,
                         target_col="logClosePrice",
                         drop_cols=["ClosePrice"])
print(res["groups"])
print(f"R²: {res['r2']:.4f}")
print(f"MdAPE (%): {res['mdape']:.2f}")


{'low_card_ohe': ['ViewYN', 'PoolPrivateYN', 'AssociationFeeFrequency', 'AttachedGarageYN', 'StateOrProvince', 'FireplaceYN', 'NewConstructionYN', 'Levels_final', 'WoodYN', 'CarpetYN', 'LaminateYN', 'VinylYN', 'TileYN', 'BambooYN', 'ConcreteYN', 'StoneYN', 'SeeRemarksYN', 'BrickYN'], 'mid_card_te': ['BuyerAgentAOR', 'ListAgentAOR', 'CloseDate', 'CountyOrParish', 'BuyerOfficeAOR', 'ContractStatusChangeDate'], 'high_card_te': ['ListAgentFirstName', 'ListAgentLastName', 'UnparsedAddress', 'ListOfficeName', 'BuyerOfficeName', 'CoListOfficeName', 'ListAgentFullName', 'CoListAgentFirstName', 'CoListAgentLastName', 'BuyerAgentMlsId', 'BuyerAgentFirstName', 'BuyerAgentLastName', 'MLSAreaMajor', 'SubdivisionName', 'City', 'PurchaseContractDate', 'ListingContractDate', 'HighSchoolDistrict', 'PostalCode', 'EmailDomain']}
R²: 0.1315
MdAPE (%): 33.38
